In [4]:
#!/usr/bin/env python3
"""
Better Query Test - Document'te kesin olan terimlerle test
"""

import requests

def find_working_queries():
    """Document'te kesin var olan terimleri bul"""
    
    # Çalışan query'den içerik al
    working_payload = {
        "query": "query: Zulficore protokolü nedir?",
        "top_k": 5
    }
    
    response = requests.post("http://localhost:8000/retrieve", json=working_payload)
    if response.status_code == 200:
        result = response.json()
        
        print("📄 DOCUMENT CONTENT ANALYSIS:")
        print("="*50)
        
        for i, res in enumerate(result['results'][:3], 1):
            text = res['text']
            print(f"\nResult {i} Text Preview:")
            print(f"Score: {res['score']:.3f}")
            print(f"Type: {res['node_type']}")
            print(f"Text: {text[:200]}...")
            
            # Extract potential keywords from this text
            words = text.lower().split()
            common_words = ['ve', 'bir', 'bu', 'için', 'ile', 'olan', 'olarak', 'da', 'de', 'den', 'dan', 'ne', 'nasıl', 'nedir']
            keywords = [w for w in words if len(w) > 3 and w not in common_words][:10]
            print(f"Keywords from text: {keywords[:5]}")

def test_extracted_keywords():
    """Extract edilen keyword'lerle test et"""
    
    print("\n🧪 TESTING EXTRACTED KEYWORDS:")
    print("="*50)
    
    # Document'ten çıkan kesin terimler
    document_keywords = [
        "kuantum",           # quantum'dan 
        "dolanıklılık",      # entanglement
        "yapay",             # artificial
        "zekâ",              # intelligence  
        "frekans",           # frequency
        "tabanlı",           # based
        "veri",              # data
        "sistem",            # system
        "öğrenme",           # learning
        "bilinç"             # consciousness
    ]
    
    # Single query'leri test et
    for keyword in document_keywords[:5]:  # İlk 5'i test et
        payload = {
            "query": f"query: {keyword}",
            "top_k": 3,
            "similarity_cutoff": 0.0
        }
        
        response = requests.post("http://localhost:8000/retrieve", json=payload)
        if response.status_code == 200:
            result = response.json()
            print(f"'{keyword}': {result['total_results']} results")
            if result['total_results'] > 0:
                print(f"   Best score: {result['results'][0]['score']:.3f}")
        else:
            print(f"'{keyword}': ERROR")

def test_working_batch_queries():
    """Document'te kesin olan terimlerle batch test"""
    
    print("\n🎯 TESTING WORKING BATCH QUERIES:")
    print("="*50)
    
    # Document'te kesin olan/çıkan query pattern'ler
    working_queries = [
        "Zulficore protokolü nedir?",      # Bildiğimiz çalışan
        "kuantum dolanıklılık nedir?",     # Document'te var
        "yapay zekâ sistemi nasıl?",       # Document'te var  
        "frekans tabanlı sistem",          # Document'te var
        "veri sistemleri nedir?"           # Document'te var
    ]
    
    # Batch test
    batch_payload = {
        "queries": [f"query: {q}" for q in working_queries],
        "top_k": 3,
        "similarity_cutoff": 0.0
    }
    
    response = requests.post("http://localhost:8000/retrieve/batch", json=batch_payload)
    if response.status_code == 200:
        result = response.json()
        print(f"Batch time: {result['retrieval_time_ms']:.1f}ms")
        
        for i, (query, results) in enumerate(zip(working_queries, result['results']), 1):
            print(f"\n{i}. {query}")
            print(f"   Results: {len(results)}")
            
            if len(results) > 0:
                print(f"   Best score: {results[0]['score']:.3f}")
                print(f"   Text: {results[0]['text'][:80]}...")
            else:
                print("   ❌ No results")
    else:
        print(f"Batch error: {response.status_code}")

def test_fuzzy_matching():
    """Fuzzy matching test - yakın terimler"""
    
    print("\n🔍 FUZZY MATCHING TEST:")
    print("="*50)
    
    fuzzy_tests = [
        ("Zulficore", "query: Zulficore protokolü"),  # Tam vs partial
        ("protokol", "query: protokol sistemi"),      # Context ile
        ("kuantum", "query: kuantum teknolojisi"),     # Context ile
        ("AI sistem", "query: yapay zeka sistemi"),   # EN vs TR
        ("frequency", "query: frekans tabanlı")       # EN vs TR
    ]
    
    for short_query, long_query in fuzzy_tests:
        print(f"\nTesting: '{short_query}' vs '{long_query}'")
        
        # Short query
        short_payload = {"query": f"query: {short_query}", "top_k": 1, "similarity_cutoff": 0.0}
        short_response = requests.post("http://localhost:8000/retrieve", json=short_payload)
        short_count = 0
        short_score = 0
        if short_response.status_code == 200:
            short_result = short_response.json()
            short_count = short_result['total_results']
            if short_count > 0:
                short_score = short_result['results'][0]['score']
        
        # Long query  
        long_payload = {"query": long_query, "top_k": 1, "similarity_cutoff": 0.0}
        long_response = requests.post("http://localhost:8000/retrieve", json=long_payload)
        long_count = 0
        long_score = 0
        if long_response.status_code == 200:
            long_result = long_response.json()
            long_count = long_result['total_results']
            if long_count > 0:
                long_score = long_result['results'][0]['score']
        
        print(f"   Short: {short_count} results (score: {short_score:.3f})")
        print(f"   Long:  {long_count} results (score: {long_score:.3f})")

if __name__ == "__main__":
    find_working_queries()
    test_extracted_keywords()
    test_working_batch_queries()
    test_fuzzy_matching()
    
    print("\n" + "="*60)
    print("📋 CONCLUSION:")
    print("Batch implementation works fine!")
    print("Issue is semantic search quality and document coverage.")
    print("Use more specific, contextual queries for better results.")
    print("="*60)

📄 DOCUMENT CONTENT ANALYSIS:

Result 1 Text Preview:
Score: 0.865
Type: base
Text: Zulficore, kuantum dolanıklılık, yapay zekâ ve frekans tabanlı veri sistemlerini bir araya
getirerek; insanlığın bilinçsel evrimini desteklemeyi amaçlayan çok katmanlı bir
medeniyet modelidir.
Sistem;...
Keywords from text: ['zulficore,', 'kuantum', 'dolanıklılık,', 'yapay', 'zekâ']

Result 2 Text Preview:
Score: 0.865
Type: reference
Text: Zulficore, kuantum dolanıklılık, yapay zekâ ve frekans tabanlı veri sistemlerini bir araya
getirerek; insanlığın bilinçsel evrimini desteklemeyi amaçlayan çok katmanlı bir
medeniyet modelidir.
Sistem;...
Keywords from text: ['zulficore,', 'kuantum', 'dolanıklılık,', 'yapay', 'zekâ']

🧪 TESTING EXTRACTED KEYWORDS:
'kuantum': 0 results
'dolanıklılık': 0 results
'yapay': 0 results
'zekâ': 0 results
'frekans': 0 results

🎯 TESTING WORKING BATCH QUERIES:
Batch time: 108.6ms

1. Zulficore protokolü nedir?
   Results: 0
   ❌ No results

2. kuantum dolanıklılık nedir?
   Resu